# PREPARACION DE LOS DATOS

## Importacion de librerias necesarias

In [1]:
# Importar Librerias
import os
import numpy as np
import pandas as pd
import cv2
import gc
from tqdm import tqdm
from glob import glob

## Pasos 1 y 2:

- Recolectar todos los datos
- Etiquetar

In [4]:
dirs = os.listdir('data')
images_path = []
labels = []
for folder in dirs:
    path = glob('./Data/{}/*.jpg'.format(folder))
    label = ['{}'.format(folder)]*len(path)
    # Anexar
    images_path.extend(path)
    labels.extend(label)

## Pasos 3 y 4: 
- Deteccion de rostros
- Recorte de la imagen

In [5]:
img_path = images_path[1]
img = cv2.imread(img_path)

In [6]:
cv2.imshow('original', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
# deteccion de rostros
face_deteccion_model = cv2.dnn.readNetFromCaffe('./Models/deploy.prototxt.txt',
                                                './Models/res10_300x300_ssd_iter_140000_fp16.caffemodel')

In [8]:
def face_detection_dnn(img):
    # blob desde imagen (rgb media de substraccion de imagen)
    image = img.copy()
    h,w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,117,123),swapRB=True)
    # Obtener las detecciones
    face_deteccion_model.setInput(blob)
    detections = face_deteccion_model.forward()
    for i in range(0, detections.shape[2]):
        confidence = detections[0,0,i,2] # Puntuacion de confianza

        if confidence > 0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            box = box.astype(int)
            # print(box)
            pt1 = (box[0],box[1])
            pt2 = (box[2],box[3])
            # cv2.rectangle(image,pt1,pt2,(0,255,0),2)
            roi = image[box[1]:box[3],box[0]:box[2]]
            
            return roi
        return None

In [9]:
img_roi = face_detection_dnn(img)

In [10]:
cv2.imshow('roi', img_roi)
cv2.imshow('original', img)
cv2.waitKey()
cv2.destroyAllWindows()

## Paso 5
- Procesamiento de imagenes
- Blob desde imagen (Subtraccion media de RGB)


In [11]:
def datapreprocess(img):
    face = face_detection_dnn(img)
    if face is not None:
        # computing blob from image
        blob = cv2.dnn.blobFromImage(face,1,(100,100),(104,117,123),swapRB=True)
        blob_squeeze = np.squeeze(blob).T
        blob_rotate = cv2.rotate(blob_squeeze,cv2.ROTATE_90_CLOCKWISE)
        blob_flip = cv2.flip(blob_rotate,1)
        # Remover valores negativos
        img_norm = np.maximum(blob_flip,0)/blob_flip.max()
    
        return img_norm
    else:
        return None


## Paso 6:
- Aplicar cambios a todas las imagenes y agregarlas en una lista

In [12]:
# len(images_path)
# ATENCION!!! --> La ejecucion de este proceso tardara una media de 1 o 2 horas asi que debo 
#                 asegurarame de no tener ningun otro programa abierto durante la ejecucion de esta celda.

data_img = []
label_img =[]
i=0
for path, label in tqdm(zip(images_path, labels),desc='Pre-procesamiento'):
    img = cv2.imread(path) # leer las rutas de las imagenes
    process_img = datapreprocess(img)
    if process_img is not None:
        data_img.append(process_img)
        label_img.append(label)
        
    i += 1
    if i%100 == 0:
        gc.collect() #Recolectar la 'basura'

Pre-procesamiento: 8000it [05:56, 22.42it/s]


## Paso 7:
- Guardar las imagenes pre-procesadas en un zip Numpy (.npz)

In [13]:
# ATENCION!!! --> NO EJECUTAR esta celda y la siguiente a menos que la anterior haya completado su proceso de ejecucion
X = np.array(data_img)
y = np.array(label_img)

In [14]:
np.savez('./Data/data_preprocesada_v1.npz',X,y)